In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import skimage.io as io
import os


## Dense
برای پیاده سازی حالت دنس مطابق راهنمای خود اوپن سیوی عمل میکنیم.ابتدا تفاوت دو فریم را محاسبه کرده،سپس اندازه و زاویه انرا در نظر میگیریم و انرا به فضای اچ اس وی میبریم و در ان فضا شارنوری را نمایش میدهیم.

In [ ]:
cap = cv2.VideoCapture('test.mp4')
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2', bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    prvs = next
cv2.destroyAllWindows()

## Sparse
برای این حالت ابتدا یک سری نقاط کلیدی استخراج میکنیم، سپس این نقاط را در تصویر رصد میکنیم و خطوط واصل را در طول مسیر برای انها رسم میکنیم.

In [ ]:
cap = cv2.VideoCapture('test.mp4')

feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)
    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv2.destroyAllWindows()
cap.release()

## Part3 
برای این بخش ابتدا داده ها رو میخوانیم و در حین خواندن محدوده صورت انها را استخراج میکنیم.
پس از اینکه تمام تصاویر یک ویدئو خوانده شد هیستوگرام انهارا متناسب با توضیحات استخراج میکنیم و انرا به طبقه بند خود میدهیم.
متاسفانه دقتی بیش از 66 درصد در این آزمایش حاصل نشد که همانطور که در کلاس هم با تی ای ها بحث شد علت پیدا نگردید:( ل

In [3]:
hist_happy = []
path = 'happy videos'
counter = 0
y=[]
for file in os.listdir(path):
    images = [] 
    for image in os.listdir(path+'/'+file):
        img= io.imread(path+'/'+file+'/'+image, as_gray=True)[150:700,380:820]
        images.append(img) 
        
    n = len(images)
    hists=[]
    for i in range(n-1):
        
        flow = cv2.calcOpticalFlowFarneback(images[i],images[i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        bins = np.linspace(0,2*np.pi,9)
        hist= np.histogram(ang.ravel(), bins=bins)[0]
        hists.append(hist)
    hists = np.array(hists)
    print(hists.shape)
    res = np.concatenate((np.mean(hists, axis=0), np.max(hists, axis=0)))
    hist_happy.append(res)                    
    y.append(1)

(14, 8)
(12, 8)
(16, 8)
(32, 8)
(12, 8)
(15, 8)
(17, 8)
(20, 8)
(16, 8)
(22, 8)
(32, 8)
(25, 8)
(22, 8)
(24, 8)
(27, 8)
(11, 8)
(21, 8)
(16, 8)
(16, 8)
(20, 8)
(21, 8)
(28, 8)
(42, 8)
(24, 8)
(18, 8)
(32, 8)
(26, 8)
(25, 8)
(12, 8)
(18, 8)


In [4]:
hist_happy = np.array(hist_happy)
hist_happy.shape

(30, 16)

In [5]:
hist_surprise = []
path = 'surprise videos'
counter = 0

for file in os.listdir(path):
    images = [] 
    for image in os.listdir(path+'/'+file):
        img= io.imread(path+'/'+file+'/'+image, as_gray=True)[150:700,380:820]
        images.append(img) 
        
    n = len(images)
    hists=[]
    for i in range(n-1):
        flow = cv2.calcOpticalFlowFarneback(images[i],images[i+1], None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        bins = np.linspace(0,2*np.pi,9)
        hist= np.histogram(ang.ravel(), bins=bins)[0]
        hists.append(hist)
    hists = np.array(hists)
    print(hists.shape)
    res = np.concatenate((np.mean(hists, axis=0), np.max(hists, axis=0)))
    hist_surprise.append(res)     
    y.append(0)
hist_surprise = np.array(hist_surprise)
hist_surprise.shape

(13, 8)
(15, 8)
(28, 8)
(29, 8)
(14, 8)
(21, 8)
(28, 8)
(14, 8)
(19, 8)
(18, 8)
(23, 8)
(5, 8)
(17, 8)
(14, 8)
(22, 8)
(118, 8)
(18, 8)
(19, 8)
(17, 8)
(11, 8)
(15, 8)
(12, 8)
(11, 8)
(21, 8)
(21, 8)
(13, 8)
(13, 8)
(15, 8)
(12, 8)
(7, 8)


(30, 16)

In [16]:


X = np.concatenate((hist_happy, hist_surprise))
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    np.array(y),
                                                    train_size=0.7,
                                                    random_state=50)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

scaler.fit(X_test)
X_test = scaler.transform(X_test)
print(X_train.shape , y_train.shape)
print(X_test.shape , y_test.shape)

(42, 16) (42,)
(18, 16) (18,)


In [11]:
mySVM = svm.SVC(C=100)
mySVM.fit(X_train, y_train.ravel())
y_pred = mySVM.predict(X_test)
Acc = accuracy_score(y_test, y_pred)

print(y_pred.shape , y_test.shape)
print('acc=',Acc)
print( y_test - y_pred)

(18,) (18,)
acc= 0.6666666666666666
[ 0  0  0 -1  0 -1  0  0  0  0 -1 -1  1 -1  0  0  0  0]
